In [1]:
# We will create a movie recommendation system

import os
import pandas as pd

# We will use the MovieLens dataset which contains user ratings for movies
ratings_filename = os.path.join('./data/', "u.data")

In [2]:
# Read CSV input with user ratings
all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])

# The full u data set: 100000 ratings by 943 users on 1682 items.
# Each user has rated many movies (at least 20)
# Users and items are numbered consecutively from 1.  
# The data is randomly ordered. 
# This is a tab separated list of: 
# user id | item id | rating | timestamp. 

print(f'Shape: {all_ratings.shape}')

# Sample:
all_ratings.head()

Shape: (100000, 4)


,UserID,MovieID,Rating,Datetime
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
# Transform last attribute to datetime
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'], unit='s')
all_ratings.head()

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [4]:
# Create an additional attribute that we will call "Favorable"
# This will be true if the user gave a rating above 3

all_ratings["Favorable"] = all_ratings["Rating"] > 3
all_ratings.head()

,UserID,MovieID,Rating,Datetime,Favorable
0,196,242,3,1997-12-04 15:55:49,False
1,186,302,3,1998-04-04 19:22:22,False
2,22,377,1,1997-11-07 07:18:36,False
3,244,51,2,1997-11-27 05:02:03,False
4,166,346,1,1998-02-02 05:33:16,False


In [5]:
# Let's use only a small sample of 200 users
# I.e. we will use the ratings of those users (not 200 samples)

ratings = all_ratings[all_ratings['UserID'].isin(range(200))]
ratings

,UserID,MovieID,Rating,Datetime,Favorable
0,196,242,3,1997-12-04 15:55:49,False
1,186,302,3,1998-04-04 19:22:22,False
2,22,377,1,1997-11-07 07:18:36,False
4,166,346,1,1998-02-02 05:33:16,False
6,115,265,2,1997-12-03 17:51:28,False
...,...,...,...,...,...
99951,130,121,5,1997-10-07 18:59:06,True
99959,193,690,4,1998-03-05 18:40:21,True
99978,113,975,5,1997-10-04 03:40:24,True
99998,13,225,2,1997-12-17 22:52:36,False


In [6]:
# Out of the above sample, get the Favorable ratings

favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings
# about half (more or less..)

,UserID,MovieID,Rating,Datetime,Favorable
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True
28,63,277,4,1997-10-01 23:10:01,True
...,...,...,...,...,...
99848,5,174,5,1997-09-30 16:15:30,True
99950,130,93,5,1997-09-22 18:41:05,True
99951,130,121,5,1997-10-07 18:59:06,True
99959,193,690,4,1998-03-05 18:40:21,True


In [22]:
# Create a dictionary with the favorable movie IDs for each user
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("UserID")["MovieID"])

# Example: favorable movies for user 1
print(f'Favorable movies by user 1:')
favorable_reviews_by_users[1]

Favorable movies by user 1:


frozenset({1,
           3,
           6,
           7,
           9,
           12,
           13,
           14,
           15,
           16,
           18,
           19,
           20,
           22,
           23,
           25,
           28,
           32,
           33,
           39,
           42,
           43,
           44,
           45,
           46,
           47,
           48,
           50,
           51,
           52,
           55,
           56,
           57,
           58,
           59,
           60,
           61,
           64,
           65,
           66,
           68,
           72,
           75,
           76,
           77,
           79,
           80,
           81,
           82,
           84,
           86,
           87,
           88,
           89,
           90,
           91,
           93,
           95,
           96,
           98,
           100,
           106,
           107,
           108,
           109,
           111,
         

In [23]:
# Sum favorable movies and group by movie ID
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie

,Favorable
MovieID,
1,66
2,5
3,4
4,21
5,6
...,...
1416,0
1417,0
1418,1


In [24]:
# Sort the top favorable movies
num_favorable_by_movie.sort_values(by="Favorable", ascending=False).head()

,Favorable
MovieID,
50,100
100,89
258,83
181,79
174,74


In [25]:
import sys
from collections import defaultdict


# Let's find some frequent itemsets with a minimum support = 50
# The following dict will be the placeholder, where the key is the K order.
# I.e. frequent_itemsets[1]: the 1-itemsets, frequent_itemsets[2]: the 2-itemsets
# For each k, we will keep a dict with the movie ID and the count
# E.g. frequent_itemsets[1] = {1: 66, 7: 67, ...}

frequent_itemsets = {}
min_support = 50

# # k=1 candidates are the movies with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"])
                            for movie_id, row in num_favorable_by_movie.iterrows()
                            if row["Favorable"] > min_support)

# 1-itemset with movie ids with more than 50 favs:
print(f'There are {len(frequent_itemsets[1])} movies with more than {min_support} favorable reviews')
frequent_itemsets[1]

There are 16 movies with more than 50 favorable reviews


{frozenset({1}): 66,
 frozenset({7}): 67,
 frozenset({9}): 53,
 frozenset({50}): 100,
 frozenset({56}): 67,
 frozenset({64}): 58,
 frozenset({79}): 58,
 frozenset({98}): 70,
 frozenset({100}): 89,
 frozenset({127}): 70,
 frozenset({172}): 59,
 frozenset({174}): 74,
 frozenset({181}): 79,
 frozenset({258}): 83,
 frozenset({286}): 59,
 frozenset({313}): 60}

In [26]:
# Create a method to find frequent itemsets of length k,
# using the frequent itemsets of length k-1

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    """
    favorable_reviews_by_users: the dict with the movie id favs by user
    k_1_itemsets: the k-1 itemsets (of previous steps)
    min_support: minimum support threshold
    
    Logic:
    for each user and their reviews:
      for each k-1 itemset (from previous steps)
        if the k-1 itemset is a subset of the current review set
          then subtract it from the current review set
          and for the the remaining movies of the current review set:
           create a superset with the previous itemset and the current movie in the loop 
           increase counts for that superset
    if corresponding counts >= min_suuport, return the superset
    """
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():      # reviews: set of movie ids
        for itemset in k_1_itemsets:                              # itemset: a k-1 itemset
            if itemset.issubset(reviews):                         # if the k-1 itemset is subset of current review set
                for other_reviewed_movie in reviews - itemset:    # create a new set k: (the k-1 itemset plus
                    current_superset = itemset | frozenset((other_reviewed_movie,))    # the current review movie)
                    counts[current_superset] += 1
    # Return the k itemsets with at least minimum support
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [27]:
sys.stdout.flush()

# So, let's generate candidates of length k, using the itemsets of length k-1
# We already have the k=1 itemset from above (frequent_itemsets[1])
# Loop for k=2 and above and store only the frequent itemsets

for k in range(2, 20):
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1], min_support)
    if len(cur_frequent_itemsets) == 0:
        print(f'Did not find any frequent itemsets of length {k}')
        sys.stdout.flush()
        break
    else:
        print(f'Found {len(cur_frequent_itemsets)} frequent itemsets of length {k}')
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
        
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

Found 93 frequent itemsets of length 2
Found 295 frequent itemsets of length 3
Found 593 frequent itemsets of length 4
Found 785 frequent itemsets of length 5
Found 677 frequent itemsets of length 6
Found 373 frequent itemsets of length 7
Found 126 frequent itemsets of length 8
Found 24 frequent itemsets of length 9
Found 2 frequent itemsets of length 10
Did not find any frequent itemsets of length 11


In [29]:
# Let's see for example the K=2 itemsets
frequent_itemsets[2]

{frozenset({1, 7}): 62,
 frozenset({1, 50}): 100,
 frozenset({1, 56}): 64,
 frozenset({1, 64}): 60,
 frozenset({1, 79}): 62,
 frozenset({1, 98}): 72,
 frozenset({1, 100}): 84,
 frozenset({1, 127}): 66,
 frozenset({1, 172}): 60,
 frozenset({1, 174}): 82,
 frozenset({1, 181}): 76,
 frozenset({7, 9}): 50,
 frozenset({7, 50}): 94,
 frozenset({7, 56}): 78,
 frozenset({7, 64}): 64,
 frozenset({7, 79}): 68,
 frozenset({7, 98}): 74,
 frozenset({7, 100}): 94,
 frozenset({7, 127}): 58,
 frozenset({7, 172}): 70,
 frozenset({7, 174}): 78,
 frozenset({7, 181}): 72,
 frozenset({7, 258}): 68,
 frozenset({9, 50}): 70,
 frozenset({9, 56}): 60,
 frozenset({9, 64}): 58,
 frozenset({9, 98}): 58,
 frozenset({9, 100}): 78,
 frozenset({9, 127}): 66,
 frozenset({9, 174}): 54,
 frozenset({9, 181}): 52,
 frozenset({50, 56}): 92,
 frozenset({50, 64}): 82,
 frozenset({50, 79}): 84,
 frozenset({50, 98}): 96,
 frozenset({50, 100}): 110,
 frozenset({50, 127}): 102,
 frozenset({50, 172}): 108,
 frozenset({50, 174}): 

In [30]:
# Now we create the association rules. 
# They are candidates until a confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

# Sample:
print('A few samples:')
print(candidate_rules[:10])

There are 15285 candidate rules
A few samples:
[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56), (frozenset({56}), 1), (frozenset({1}), 64), (frozenset({64}), 1), (frozenset({79}), 1), (frozenset({1}), 79)]


In [31]:
# Let's compute the confidence of each rule

# Just search for rule premise and rule conclusion in the reviews dataset
# and keep counts in order to calculate the confidence for each rule down the line
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [32]:
# Choose only rules above a minimum confidence level
# and filter out the rules with "poor" confidence

min_confidence = 0.9
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}

rule_confidence

{(frozenset({172}), 50): 0.9152542372881356,
 (frozenset({1, 172}), 50): 0.9666666666666667,
 (frozenset({1, 181}), 50): 0.9473684210526315,
 (frozenset({1, 56}), 174): 0.90625,
 (frozenset({7, 174}), 50): 0.9230769230769231,
 (frozenset({7, 181}), 50): 0.9444444444444444,
 (frozenset({7, 172}), 174): 0.9714285714285714,
 (frozenset({9, 181}), 50): 0.9615384615384616,
 (frozenset({9, 174}), 56): 0.9259259259259259,
 (frozenset({56, 172}), 50): 0.918918918918919,
 (frozenset({50, 56}), 174): 0.9565217391304348,
 (frozenset({56, 181}), 50): 0.9705882352941176,
 (frozenset({64, 172}), 50): 0.9393939393939394,
 (frozenset({64, 181}), 50): 0.9705882352941176,
 (frozenset({79, 172}), 50): 0.9411764705882353,
 (frozenset({50, 79}), 174): 0.9523809523809523,
 (frozenset({79, 181}), 50): 0.9714285714285714,
 (frozenset({98, 181}), 50): 1.0,
 (frozenset({100, 181}), 50): 0.9230769230769231,
 (frozenset({127, 172}), 50): 0.9666666666666667,
 (frozenset({172, 174}), 50): 0.9423076923076923,
 (froz

In [33]:
# Sort
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

# Let's print a few top rules
for index in range(5):
    print(f'Rule #{index+1}')
    (premise, conclusion) = sorted_confidence[index][0]
    print(f'If a person recommends {premise} they will also recommend {conclusion}')
    print(f' - Confidence: {rule_confidence[(premise, conclusion)]}')
    print('')


Rule #1
If a person recommends frozenset({98, 181}) they will also recommend 50
 - Confidence: 1.0

Rule #2
If a person recommends frozenset({172, 79}) they will also recommend 174
 - Confidence: 1.0

Rule #3
If a person recommends frozenset({258, 172}) they will also recommend 174
 - Confidence: 1.0

Rule #4
If a person recommends frozenset({1, 181, 7}) they will also recommend 50
 - Confidence: 1.0

Rule #5
If a person recommends frozenset({1, 172, 7}) they will also recommend 174
 - Confidence: 1.0



In [34]:
# There is an additional file in the dataset that provides the movie info

# -- Information about the (movies): tab separated list of
#    movie id | movie title | release date | video release date |
#    IMDb URL | unknown | Action | Adventure | Animation |
#    Children's | Comedy | Crime | Documentary | Drama | Fantasy |
#    Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
#    Thriller | War | Western |
# -- The last 19 fields are the genres, a 1 indicates the movie
#    is of that genre, a 0 indicates it is not; movies can be in
#    several genres at once. The movie ids are the ones used in the u.data data set.

movie_name_filename = os.path.join('./data/', "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

# Sample
movie_name_data.head()

,MovieID,Title,Release Date,Video Release,IMDB,<UNK>,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [35]:
# Let's print the same rules by using the movie name

# Define a function to get movie name
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

# Print the rules again using the movie title this time
for index in range(5):
    print(f'Rule #{index+1}')
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print(f'If a person recommends {premise_names} they will also recommend {conclusion_name}')
    print(f' - Confidence: {rule_confidence[(premise, conclusion)]}')
    print('')

Rule #1
If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Confidence: 1.0

Rule #2
If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.0

Rule #3
If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.0

Rule #4
If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Confidence: 1.0

Rule #5
If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Confidence: 1.0



In [36]:
######################

In [37]:
# Let's check the same rules for the remaining of the original dataset

# So, the remaining dataset, i.e. the one that contains the reviews from the rest of the users (with ID>=200)
# we will call it test set
test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
# And again we will get the favorable reviews
test_favorable = test_dataset[test_dataset["Favorable"]]
# And again we will create a dict for the favs for each user
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("UserID")["MovieID"])

In [38]:
# And we will calculate the confidence of the rules that we have already generated above
# in this dataset now

correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

test_confidence = {candidate_rule:
                   (correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]))
                   for candidate_rule in rule_confidence}

test_confidence

{(frozenset({172}), 50): 0.9401709401709402,
 (frozenset({1, 172}), 50): 0.944,
 (frozenset({1, 181}), 50): 0.9371069182389937,
 (frozenset({1, 56}), 174): 0.7722772277227723,
 (frozenset({7, 174}), 50): 0.8490566037735849,
 (frozenset({7, 181}), 50): 0.9380530973451328,
 (frozenset({7, 172}), 174): 0.8888888888888888,
 (frozenset({9, 181}), 50): 0.9041095890410958,
 (frozenset({9, 174}), 56): 0.75,
 (frozenset({56, 172}), 50): 0.9624060150375939,
 (frozenset({50, 56}), 174): 0.7719298245614035,
 (frozenset({56, 181}), 50): 0.9672131147540983,
 (frozenset({64, 172}), 50): 0.95,
 (frozenset({64, 181}), 50): 0.9416666666666667,
 (frozenset({79, 172}), 50): 0.9854014598540146,
 (frozenset({50, 79}), 174): 0.8323699421965318,
 (frozenset({79, 181}), 50): 0.9621212121212122,
 (frozenset({98, 181}), 50): 0.9358974358974359,
 (frozenset({100, 181}), 50): 0.9177215189873418,
 (frozenset({127, 172}), 50): 0.957983193277311,
 (frozenset({172, 174}), 50): 0.9574468085106383,
 (frozenset({50, 172}

In [39]:
# And finally, let's print the top rules again reporting also the confidence in the test set
for index in range(5):
    print(f'Rule #{index+1}')
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print(f'If a person recommends {premise_names} they will also recommend {conclusion_name}')
    print(f' - Train Confidence: {rule_confidence[(premise, conclusion)]}')
    print(f' - Test Confidence: {test_confidence[(premise, conclusion)]}')
    print('')

Rule #1
If a person recommends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommend Star Wars (1977)
 - Train Confidence: 1.0
 - Test Confidence: 0.9358974358974359

Rule #2
If a person recommends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.0
 - Test Confidence: 0.8759124087591241

Rule #3
If a person recommends Contact (1997), Empire Strikes Back, The (1980) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.0
 - Test Confidence: 0.8409090909090909

Rule #4
If a person recommends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommend Star Wars (1977)
 - Train Confidence: 1.0
 - Test Confidence: 0.9324324324324325

Rule #5
If a person recommends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommend Raiders of the Lost Ark (1981)
 - Train Confidence: 1.0
 - Tes